In [1]:
import pygame
import pygame_menu
import pygame.freetype
import sys
import random
import math
import numpy as np

from gym import Env
from gym.spaces import Discrete, Box

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

pygame 2.0.1 (SDL 2.0.14, Python 3.6.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import sys
import os
#os.chdir(r"..\\")
#Import local modules
print(os.getcwd())
sys.path.append(r"C:/Users/chris/OneDrive/Documents/Masters Project")
from ipynb.fs.full.Games.SpaceInvaders import Space_Invaders
from ipynb.fs.full.Games.Asteroids import Asteroids

pygame.init()

C:\Users\chris\OneDrive\Documents\Masters Project


(8, 0)

In [3]:
#checks if path is a file
#isFile = os.path.isfile(r"C:/Users/chris/OneDrive/Documents/Masters Project/Dependencies/Resources/Masterslogo.png")

#checks if path is a directory
#isDirectory = os.path.isdir(r"C:/Users/chris/OneDrive/Documents/Masters Project/Dependencies/Resources")

#actual = os.path.join(os.getcwd(), "Masters Project/Dependencies/Resources", "Masterslogo.png")
#print(os.path.isfile(actual))
#print(actual)
#print(isFile, isDirectory)

In [4]:
class Custom_OpenAI_Env(Env):
    def __init__(self, screen_width, screen_height, screen, surface, game, 
                 colour = False, action_space = Discrete(4)):
        # Actions we can take; left, right, shoot, no_action
        self.action_space = action_space# Discrete(4)
        # Array of observation, dont know how to implement this as image based input.
        self.observation_space = Box(0, 255, shape=(1, screen_width, screen_height, ))
        #(screen_height * screen_width * 3,))# Box(low=0, high=255, 
                                            #shape=(screen_height * screen_width, 1), dtype=np.uint8)
        print("observation space shape = ", self.observation_space.shape)
        #Initialise pygame
        self.game = game# Space_Invaders(screen_width, screen_height, screen, surface)
        print("game set")
        # Initial state should be image of the first frame of the game
        self.colour = colour
        print("color set")
        self.state = self.game.get_state(colour = self.colour)
        print("retrieved state")
        self.start_state = self.state
        
        self.surface = surface
        self.screen = screen
        print("initialisation complete")
        
        
    def step(self, action):
        # Apply action
        self.game.execute_action(action)
        
        #Call the update loop before getting the state
        self.game.update()
        self.state = self.game.get_state(colour = self.colour)
        #print("state shape: ", self.state.shape)
        # Calculate reward
        #Calculated as follows: if player is still alive, +0.001 (perhaps), if player is dead, -1
        #increasing function dependant on how many aliens are left (the less there are the higher
        #the value from value of 0 - 1 with 0 being all still alive, 1 being all dead, game over)
        reward = self.game.calculate_reward()
        
        # Check if shower is done
        #if function equals 1, return done
        done = self.game.done
            
        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info
    
    def render(self, mode):
        #Put render loop in here
        self.game.render(mode=mode)
    def reset(self):
        #Restart the game
        self.state = self.start_state
        self.game.reset()
        return self.game.get_state()
    

In [5]:
#Game menu class controlling the functionality of the entire framework
class Game_Menu:

    def __init__(self, width, height):
        self.screen = None
        self.menu = None
        self.surface = None
        self.game = None
        self.icon_surface = None
        
        #Set standard network parameters
        self.learning_rates = [0.1, 0.01, 0.001]
        self.episodes = [1, 5, 10]
        self.steps = [1000, 5000, 10000]

        #Standard parameter indices
        self.learning_rate = 0
        self.episode = 0
        self.step = 0
        self.visualize = False

        #Asteroid specific parameters
        self.scales = [(800,600), (400,300), (200,150)]
        self.intensity_rates = [0.1, 0.5, 1.0]
        self.player_speeds = [1, 1.5, 3]

        #Scale, player speed and colour are common to both games
        self.scale = 0
        self.intensity = 0
        self.player_speed = 0
        self.homogenous_controls = False
        self.colour = False
        
        #Space invaders specific parameters
        self.enemy_speeds = [0.5, 1.0, 2.0]
        self.enemy_speed = 0
        
        pygame.display.init()
        
        self.surface = pygame.display.set_mode((800, 600))
        
        self.icon_surface = pygame.image.load(os.path.join(os.getcwd(), "Dependencies/Resources", "Masterslogo.png"))
        pygame.display.set_icon(self.icon_surface)
        pygame.display.set_caption("Master's Project")
        
        self.main_menu()
        
    def main_menu(self):
        print("going back, main menu called")
        if self.menu:
            self.menu.disable()
        self.menu = pygame_menu.Menu(600, 800, 'Main Menu',
                         theme=pygame_menu.themes.THEME_DARK)
        print("menu created")
        self.menu.add_button('Space Invaders', self.start_space_invaders)
        self.menu.add_button('Space Invaders - Training', self.set_standard_parameters, gamemode = 1)
        self.menu.add_button('Asteroids', self.start_asteroids)
        self.menu.add_button('Asteroids - Training', self.set_standard_parameters)# self.start_asteroids_training)
        self.menu.add_button('Quit', pygame_menu.events.EXIT)
        self.menu.mainloop(self.surface)

        
    def set_standard_param(self, param):
        #Learning rate 
        if param == 0:
            if self.learning_rate < 2:
                self.learning_rate += 1
            else:
                self.learning_rate = 0
        #Steps
        elif param == 1:
            if self.step < 2:
                self.step += 1
            else:
                self.step = 0
        #Episodes
        elif param == 2:
            if self.episode < 2:
                self.episode += 1
            else:
                self.episode = 0
        #Visualize
        elif param == 3:
            self.visualize = 1 if self.visualize == 0 else 1 
            
                    
    def set_standard_parameters(self, gamemode = 0):
        print("standard params called")
        self.menu.disable()
        self.gamemode = gamemode
        self.menu = pygame_menu.Menu(600, 800, 'Select Variables',
                               theme=pygame_menu.themes.THEME_DARK)

        self.menu.add_selector('Learning rate :', [('1e-1', self.learning_rates[0]),
                                                   ('1e-2', self.learning_rates[1]),
                                                   ('1e-3', self.learning_rates[2])],
                                                   onchange=self.set_standard_param(0))
        self.menu.add_selector('Steps :', [('1000', self.steps[0]),
                                           ('5000', self.steps[1]),
                                           ('10000', self.steps[2]),],
                                           onchange=self.set_standard_param(1))
        self.menu.add_selector('Episodes: ', [('1', self.episodes[0]),
                                           ('5', self.episodes[1]),
                                           ('10', self.episodes[2]),],
                                           onchange=self.set_standard_param(2))
        self.menu.add_selector('Visualize', [('False', 0), ('True', 1)],
                               onchange=self.set_standard_param(3))
        self.menu.add_button('Back', self.main_menu)
        if gamemode == 0:
            self.menu.add_button('Continue', self.set_asteroids_parameters)
        else:
           self.menu.add_button('Continue', self.set_space_invaders_parameters)
        
        self.menu.mainloop(self.surface)
    
    def set_mode_parameter(self, param):
        #0 scale, 1 intensity, 2, player speed, 3 homo controls, 4 colours, 5 enemy speed
        if param == 0:
            self.scale += 1 if self.scale < 2 else 0
        if param == 1:
            self.intensity += 1 if self.intensity < 2 else 0        
        if param == 2:
            self.player_speed += 1 if self.player_speed < 2 else 0        
        if param == 3:
            self.homogenous_controls = True if self.homogenous_controls == False else False
        if param == 4:
            self.colour = True if self.colour == False else False
        if param == 5:
            self.enemy_speed += 1 if self.enemy_speed < 2 else 0
            
    def set_asteroids_parameters(self):
        print("calling this but shouldnt be")
        self.menu.disable()
        self.menu = pygame_menu.Menu(600, 800, 'Select Gameplay Variables',
                               theme=pygame_menu.themes.THEME_DARK)

        self.menu.add_selector('Scale :', [('(800, 600)', 1), ('(400, 300)', 2),
                                          ('(200, 150)', 3)],
                               onchange=self.set_mode_parameter(0))
        self.menu.add_selector('Intensity:', [('0.1', 1), ('0.5', 2),
                                          ('1.0', 3)],
                               onchange=self.set_mode_parameter(1))
        self.menu.add_selector('Player Speed :', [('1', 1), ('1.5', 2),
                                          ('3', 3)],
                               onchange=self.set_mode_parameter(2))
        self.menu.add_selector('Homogenous Controls: ', [('False', 1), ('True', 2)],
                        onchange=self.set_mode_parameter(3))
        self.menu.add_selector('Colour Input: ', [('False', 1), ('True', 2)],
                        onchange=self.set_mode_parameter(4))
    
        self.menu.add_button('Start Training', self.start_asteroids_training)

        self.menu.add_button('Back', self.main_menu)
        self.menu.mainloop(self.surface)
    
    def set_space_invaders_parameters(self):
        self.menu.disable()
        self.menu = pygame_menu.Menu(600, 800, 'Select Gameplay Variables',
                               theme=pygame_menu.themes.THEME_DARK)

        self.menu.add_selector('Scale :', [('(800, 600)', 1), ('(400, 300)', 2),
                                          ('(200, 150)', 3)],
                               onchange=self.set_mode_parameter(0))
        self.menu.add_selector('Enemy Speed:', [('0.5', 1), ('1.0', 2),
                                          ('2.0', 3)],
                               onchange=self.set_mode_parameter(5))
        self.menu.add_selector('Player Speed :', [('1', 1), ('1.5', 2),
                                          ('3', 3)],
                               onchange=self.set_mode_parameter(2))
        self.menu.add_selector('Colour Input: ', [('False', 1), ('True', 2)],
                        onchange=self.set_mode_parameter(4))
        
        self.menu.add_button('Start Training', self.start_asteroids_training)
        self.menu.add_button('Back', self.main_menu)
        self.menu.mainloop(self.surface)

    def start_asteroids(self):
        self.game = Asteroids(800, 600, self.screen, self.surface, False)

    
    def start_asteroids_training(self):
        self.menu.disable()
        print("starting asteroid training")
        training_game = Asteroids(800, 600, self.screen, self.surface, True, 1.0)
        #Self.game is env
        print("creating game")
        self.game = Custom_OpenAI_Env(800, 600, self.screen, self.surface, training_game, colour=True)
        print("building model")
        self.build_model()
        print("model build, training")
        self.train_model()
    
    def start_space_invaders(self):
        self.menu.disable()
        self.game = Space_Invaders(70.0, 60.0, self.screen, self.surface, False, 0.5)
        
    def start_space_invaders_training(self):
        self.menu.disable()
        training_game = Space_Invaders(350.0, 300.0, self.screen, self.surface, True, 1.0)
        #Self.game is env
        action_space= Discrete(3)
        if self.homogenous_controls:
            action_space = Discrete(5)
        self.game = Custom_OpenAI_Env(350, 300, self.screen, self.surface, training_game, colour=True,
                                     action_space = action_space)
        self.build_model()
        self.train_model()
        
    def build_model(self):
        #Initialise state/action arrays
        states = self.game.observation_space.shape
        print("states shape: ", states)
        self.actions = self.game.action_space.n
        #Initialise the DRL model
        self.model = self.build_network(states, self.actions)
        self.model.summary()
        
    def train_model(self, metrics = ['mae'], episodes = 5, verbose = 1):
        #Initialise the DQN agent
        print("parameters: ", "\nlearning rate: ", self.learning_rates[self.learning_rate],
             "\nepisodes: ",  self.episodes[self.episode],
              "\nsteps: ", self.steps[self.step],
              "\nvisualize: ", self.visualize,
              "\nscale: ", self.scales[self.scale],
              "\nintensity (asteroids only): ", self.intensity_rates[self.intensity],
              "\nplayer speed : ", self.player_speeds[self.player_speed],
              "\nhomogenous controls (space invaders only): ", self.homogenous_controls,
              "\ncolour: ", self.colour)

        
        dqn = self.build_agent(self.model, self.actions)
        dqn.compile(Adam(lr=self.learning_rates[self.learning_rate]), metrics=metrics)
        print("beginning training")
        #Fit with openAI gym
        dqn.fit(self.game, nb_steps=self.steps[self.step], visualize=self.visualize, verbose=verbose)
        print("training complete")
        #Test with scores
        scores = dqn.test(self.game, nb_episodes=self.episodes[self.episode], visualize=self.visualize)
        print(np.mean(scores.history['episode_reward']))
        self.save(self.model)
        print("model saved")
        
    def save(self, model, name='default-model'):
        #was dqn now model
        model.save_weights(name + '.h5f', overwrite=True)
        
    def load(self, model, name):
        return dqn.load_weights(name + '.h5f')
    
    def build_network(self, states, actions):
      model = Sequential()

      model.add(Conv2D(32, (23,20), activation='relu', input_shape=states, padding='same'))#(32, 32, 3)))
      model.add(MaxPooling2D((5, 5), padding = 'same'))
      model.add(Conv2D(32, (10, 10), activation='relu', padding='same'))
      model.add(MaxPooling2D((2, 2), padding='same'))
     # model.add(Conv2D(10, (2, 2), activation='relu'))
      #model.add(Dense(100, activation='relu', input_shape=states))
      model.add(Flatten())
      model.add(Dense(100, activation='relu'))
      model.add(Dense(100, activation='relu'))
      model.add(Dense(4, activation='linear'))
      print(model.summary())
      return model

    #Function to create DQN model 
    def build_agent(self, model, actions):
        policy = BoltzmannQPolicy()
        memory = SequentialMemory(limit=50000, window_length=1)
        dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                      nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
        return dqn

In [ ]:
Game = Game_Menu(800,600)

going back, main menu called
menu created
standard params called
calling this but shouldnt be
starting asteroid training
creating game
observation space shape =  (1, 800, 600)
game set
color set
retrieved state
initialisation complete
building model
states shape:  (1, 800, 600)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 800, 32)        8832032   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 160, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 160, 32)        102432    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 80, 32)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2560)